In [221]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [223]:
import numpy as np
import pandas as pd
import hydro_serving_grpc as hs_grpc
import hydro_serving_grpc.manager as hsm
from client import HydroServingClient, HydroServingServable

In [224]:
hs_client = HydroServingClient('localhost:9090')

In [225]:
servable = hs_client.deploy_servable('adult-columnar')

2019-07-08 07:30:44.545 | INFO     | client:deploy_servable:77 - adult-columnar-1-western-dew is ServableStatus.STARTING
2019-07-08 07:30:48.856 | INFO     | client:deploy_servable:77 - adult-columnar-1-western-dew is ServableStatus.SERVING


In [228]:
hs_servable = HydroServingServable(servable.name, servable.model_version, hs_client)
hs_model = hs_client.get_model('adult-columnar')

In [230]:
x = pd.DataFrame(np.random.randint(-1000, 1000, 5*12).reshape(5, 12), columns=hss.contract.input_names)
for i in range(1):
    y = hs_servable(x)
    print(y)

{'Prediction': array([[0],
       [1],
       [1],
       [1],
       [1]])}


In [232]:
x = pd.DataFrame(np.random.randint(-1000, 1000, 5*12).reshape(5, 12), columns=hss.contract.input_names)
for i in range(1):
    y = hs_model(x)
#     print(y)

_Rendezvous: <_Rendezvous of RPC that terminated with:
	status = StatusCode.INTERNAL
	details = "Can't find servable adult-columnar"
	debug_error_string = "{"created":"@1562560295.931216000","description":"Error received from peer ipv6:[::1]:9090","file":"src/core/lib/surface/call.cc","file_line":1052,"grpc_message":"Can't find servable adult-columnar","grpc_status":13}"
>

In [233]:
from hydro_serving_grpc.reqstore import reqstore_client
rs_client = reqstore_client.ReqstoreClient("127.0.0.1:9090", insecure=True)

entries = []
for r in rs_client.subsampling("2", amount=100):
    entries.extend(r.entries)
    
print(max([len(e.request.inputs['Age'].int64_val)for e in entries]))
requests = [hss.contract.decode_request(e.request) for e in entries]

2


In [235]:
# concat requsests
rs = []
order = hss.contract.input_names
for r in requests:
    columns = []
    for feature_name in order:
        columns.append( r[feature_name])
    ndarray = np.hstack(columns)
    df = pd.DataFrame(ndarray, columns=order)
    rs.append(df)
data = pd.concat(rs)

In [236]:
data

,Age,Workclass,Education,Marital Status,Occupation,Relationship,Race,Sex,Capital Gain,Capital Loss,Hours per week,Country
0,1,1,1,15050,1,1,1,1,100,1,1,1
1,23423,32432,324,234,23423,23423,23423,23423,23423,23432,32432,234234
0,1,1,1,15050,1,1,1,1,100,1,1,1
1,23423,32432,324,234,23423,23423,23423,23423,23423,23432,32432,234234
0,1,1,1,15050,1,1,1,1,100,1,1,1
1,23423,32432,324,234,23423,23423,23423,23423,23423,23432,32432,234234
0,1,1,1,1,1,1,1,1,1,1,1,1
0,1,1,1,1,1,1,1,1,1,1,1,1
0,1,1,1,1,1,1,1,1,1,1,1,1
0,1,1,1,15050,1,1,1,1,100,1,1,1
